### Screens
#### Value 
###### EV/EBITDA
#### Quality
###### Long term FCF generation
###### Long term ROIC
###### Long term margin characteristics
###### Piotroski's F-score



### TECHNICAL INDICATOR ANALYSIS
###### Momentum +
###### advanced Momentum +
###### Alpha Architect Momentum +
###### Trendline +
###### Volatility +
###### TEM +
###### MaxGap -
### More TA-Lib Indicators
###### LINEARREG_INTERCEPT +
###### MEDPRICE +
###### TRANGE +
###### TYPPRICE +
###### MINUS_DM +
###### PLUS_DM +
###### Price oscillator +
###### Mean Rev 1M -
###### Volatility 3M -
###### ADX - 
###### DX - 
### Fundamental Factors
###### FCF to EV +
###### CapEx Vol +
###### Debt to Total Assets +
###### *capex and debt to assets highly correlated
### Quantopian Pre-made
###### 'Asset Growth 3M'  
###### 'Asset to Equity Ratio'
###### 'Capex to Sales'
###### 'Debt to Asset Ratio'-
###### 'Dividend Growth'
###### 'Dividend Yield'-
###### 'EBIT to Assets'
###### 'EV to Cashflows'
###### 'Mean Reversion 1M' -
###### 'Net Income Margin'
###### 'Price Momentum 12M'
###### 'Price Oscillator'
###### 'Price to Book'
###### 'Price to Operating Cashflows'
###### 'Price to Free Cashflows'
###### 'Return on Total Assets'
###### 'Return on Total Equity'
###### 'Return on Invest Capital'
###### 'Trendline'
###### 'Vol 3M'-

In [ ]:
#quantopian imports
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import QTradableStocksUS

from quantopian.pipeline.factors import Latest
from quantopian.pipeline.data import morningstar, Fundamentals
from quantopian.pipeline.factors import CustomFactor, SimpleMovingAverage, AverageDollarVolume,SimpleBeta, Returns, RSI,EWMA
from quantopian.pipeline.classifiers.morningstar import Sector
from quantopian.pipeline.classifiers.fundamentals import Sector
from quantopian.pipeline.data.zacks import EarningsSurprises
from quantopian.pipeline.data import factset
from quantopian.pipeline.data.psychsignal import stocktwits

#Python imports
import math
import talib
import numpy as np
import pandas as pd
import pyfolio as pf
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import linear_model, decomposition, ensemble, preprocessing, isotonic, metrics
from scipy.stats.mstats import winsorize
from zipline.utils.numpy_utils import (
    repeat_first_axis,
    repeat_last_axis,
)
from scipy.stats.mstats import gmean
from sklearn.cluster import SpectralClustering
 
from collections import Counter
import talib

bs = morningstar.balance_sheet
cfs = morningstar.cash_flow_statement
is_ = morningstar.income_statement
or_ = morningstar.operation_ratios
er = morningstar.earnings_report
v = morningstar.valuation
vr = morningstar.valuation_ratios

In [ ]:
"""
HELPER FUNCTIONS
"""

def plus_dm_helper(high, low):
    """
    Returns positive directional movement. Abstracted for use with more complex factors

    https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/DMI

    Parameters
    ----------
    high : np.array
        matrix of high prices
    low : np.array
        matrix of low prices

    Returns
    -------
    np.array : matrix of positive directional movement

    """
    # get daily differences between high prices
    high_diff = (high - np.roll(high, 1, axis=0))[1:]

    # get daily differences between low prices
    low_diff = (np.roll(low, 1, axis=0) - low)[1:]

    # matrix of positive directional movement
    return np.where(((high_diff > 0) | (low_diff > 0)) & (high_diff > low_diff), high_diff, 0.)

def minus_dm_helper(high, low):
    """
    Returns negative directional movement. Abstracted for use with more complex factors

    https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/DMI

    Parameters
    ----------
    high : np.array
        matrix of high prices
    low : np.array
        matrix of low prices

    Returns
    -------
    np.array : matrix of negative directional movement

    """
    # get daily differences between high prices
    high_diff = (high - np.roll(high, 1, axis=0))[1:]

    # get daily differences between low prices
    low_diff = (np.roll(low, 1, axis=0) - low)[1:]

    # matrix of megative directional movement
    return np.where(((high_diff > 0) | (low_diff > 0)) & (high_diff < low_diff), low_diff, 0.)


def trange_helper(high, low, close):
    """
    Returns true range

    http://www.macroption.com/true-range/

    Parameters
    ----------
    high : np.array
        matrix of high prices
    low : np.array
        matrix of low prices
    close: np.array
        matrix of close prices

    Returns
    -------
    np.array : matrix of true range

    """
    # define matrices to be compared
    close = close[:-1]
    high = high[1:]
    low = low[1:]

    # matrices for comparison
    high_less_close = high - close
    close_less_low = close - low
    high_less_low = high - low

    # return maximum value for each cel
    return np.maximum(high_less_close, close_less_low, high_less_low)


ZSCORE_FILTER = 3 # Maximum number of standard deviations to include before counting as outliers
ZERO_FILTER = 0.001 # Minimum weight we allow before dropping security
"""
def preprocess(a):
    a = a.astype(np.float64)
    a[np.isinf(a)] = np.nan
    a = np.nan_to_num(a - np.nanmean(a))
    a = a.winsorize(min_percentile=0.02,
                    max_percentile=0.98,
                    mask=QTradableStocksUS() & a.isfinite())
    a= a.zscore(a)
    
    return a
"""
def preprocess(a):
    a = a.astype(np.float64)
    a[np.isinf(a)] = np.nan
    a = np.nan_to_num(a - np.nanmean(a))
    a = winsorize(a, limits=[0.02,0.98])  
    return a

In [ ]:
def make_factors_pos():
    #Momentum Indicators
    class Momentum(CustomFactor):
        inputs = [USEquityPricing.close]
        window_length = 252

        def compute(self, today, assets, out, close):       
            out[:] = close[-20] / close[0]

    class aa_momentum(CustomFactor):
        """ Alpha Architect - Momentum factor """
        inputs = [USEquityPricing.close,
        Returns(window_length=126)]
        window_length = 252
        def compute(self, today, assets, out, prices, returns):  
                out[:] = ((prices[-21] - prices[-252])/prices[-252] -  
                          (prices[-1] - prices[-21])/prices[-21]) / np.nanstd(returns, axis=0)
                
    class ATR(CustomFactor):
            """
            Average True Range

            Momentum indicator

            **Default Inputs:** USEquityPricing.high, USEquityPricing.low, USEquityPricing.close

            **Default Window Length:** 15 (14+1)

            https://en.wikipedia.org/wiki/Average_true_range
            """
            inputs=[USEquityPricing.high, USEquityPricing.low, USEquityPricing.close]
            window_length = 21

            def compute(self, today, assets, out, high, low, close):

                tr_frame = trange_helper(high, low, close)
                decay_rate= 2./(len(tr_frame) + 1.)
                weights = np.full(len(tr_frame), decay_rate, float) ** np.arange(len(tr_frame) + 1, 1, -1)
                out[:] = np.average(tr_frame, axis=0, weights=weights)
                
    class MINUS_DM(CustomFactor):
        """
        Negative directional movement

        Momentum indicator

        **Default Inputs:** USEquityPricing.high, USEquityPricing.low

        **Default Window Length:** 15

        https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/DMI
        """	

        inputs = [USEquityPricing.high, USEquityPricing.low]
        window_length = 22

        def compute(self, today, assets, out, high, low):
                out[:] = np.sum(minus_dm_helper(high, low), axis=0)


    class PLUS_DM(CustomFactor):
        """
        Positive directional movement

        Momentum indicator

        **Default Inputs:** USEquityPricing.high, USEquityPricing.low

        **Default Window Length:** 15

        https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/DMI
        """	
        inputs = [USEquityPricing.high, USEquityPricing.low]
        window_length = 22

        def compute(self, today, assets, out, high, low):
                out[:] = np.sum(plus_dm_helper(high, low), axis=0)
                
    def Price_Momentum_12M():
        """
        12-Month Price Momentum:
        12-month closing price rate of change.
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests momentum (long term)
        Equivalent to analysis of returns (12-month window)
        """
        return Returns(window_length=252)
    
    #Trend Indicators
    class Trendline(CustomFactor):
            inputs = [USEquityPricing.close]
            window_length = 252
            window_safe = True

            _x = np.arange(window_length)
            _x_var = np.var(_x)

            def compute(self, today, assets, out, close):

                x_matrix = repeat_last_axis(
                (self.window_length - 1) / 2 - self._x,
                len(assets),
                )

                y_bar = np.nanmean(close, axis=0)
                y_bars = repeat_first_axis(y_bar, self.window_length)
                y_matrix = close - y_bars

                out[:] = preprocess(-np.divide(
                (x_matrix * y_matrix).sum(axis=0) / self._x_var,
                self.window_length
                ))
                
    class LINEARREG_INTERCEPT(CustomFactor):
        """
        Intercept of Trendline

        **Default Inputs:**  USEquitypricing.close

        **Default Window Length:** 14

        http://www.stat.cmu.edu/~cshalizi/mreg/15/lectures/06/lecture-06.pdf
        """
        inputs=[USEquityPricing.close]
        window_length=21

        # using MLE
        def compute(self, today, assets, out, close):

            # prepare X matrix (x_is - x_bar)
            X = range(self.window_length)
            X_bar = np.nanmean(X)
            X_vector = X - X_bar
            X_matrix = np.tile(X_vector, (len(close.T), 1)).T

            # prepare Y vectors (y_is - y_bar)
            Y_bar = np.nanmean(close, axis=0)
            Y_bars = np.tile(Y_bar, (self.window_length, 1))
            Y_matrix = close - Y_bars

            # prepare variance of X
            X_var = np.nanvar(X)

            # multiply X matrix an Y matrix and sum (dot product)
            # then divide by variance of X
            # this gives the MLE of Beta
            betas = (np.sum((X_matrix * Y_matrix), axis=0) / X_var) / (self.window_length)

            # now use to get to MLE of alpha
            out[:] = Y_bar - (betas * X_bar)
            
    class TEM(CustomFactor):
            """
            TEM = standard deviation of past 6 quarters' reports
            """
            inputs=[factset.Fundamentals.capex_qf_asof_date,
                factset.Fundamentals.capex_qf,
                factset.Fundamentals.assets]
            window_length = 390
            window_safe = True
            def compute(self, today, assets, out, asof_date, capex, total_assets):
                values = capex/total_assets
                values[np.isinf(values)] = np.nan
                out_temp = np.zeros_like(values[-1,:])
                for column_ix in range(asof_date.shape[1]):
                    _, unique_indices = np.unique(asof_date[:, column_ix], return_index=True)
                    quarterly_values = values[unique_indices, column_ix]
                    if len(quarterly_values) < 6:
                        quarterly_values = np.hstack([
                        np.repeat([np.nan], 6 - len(quarterly_values)),
                        quarterly_values,
                        ])

                    out_temp[column_ix] = np.std(quarterly_values[-6:])

                out[:] = preprocess(-out_temp) 
                
    class PriceOscillator(CustomFactor):
        inputs = (USEquityPricing.close,)
        window_length = 252

        def compute(self, today, assets, out, close):
            four_week_period = close[-20:]
            np.divide(
                np.nanmean(four_week_period, axis=0),
                np.nanmean(close, axis=0),
                out=out,
            )
            out -= 1

    class MEDPRICE(CustomFactor):
        """
        Mean of a day's high and low prices

        **Default Inputs:**  USEquityPricing.high, USEquityPricing.low

        **Default Window Length:** 1

        http://www.fmlabs.com/reference/default.htm?url=MedianPrices.htm
        """	    
        inputs = [USEquityPricing.high, USEquityPricing.low]
        window_length = 1

        def compute(self, today, assets, out, high, low):
            out[:] = (high + low) / 2.
            
    class TYPPRICE(CustomFactor):
        """
        Typical Price 

        **Default Inputs:**  USEquityPricing.high, USEquityPricing.low, USEquityPricing.close

        **Default Window Length:** 1

        https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/typical-price
        """    
        inputs = [USEquityPricing.high, USEquityPricing.low, USEquityPricing.close]
        window_length = 1

        def compute(self, today, assets, out, high, low, close):
            out[:] = (high + low + close) / 3.


    class TRANGE(CustomFactor):
        """
        True Range 

        **Default Inputs:**  USEquityPricing.high, USEquityPricing.low, USEquityPricing.close

        **Default Window Length:** 2

        https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/atr
        """    
        inputs = [USEquityPricing.high, USEquityPricing.low, USEquityPricing.close]
        window_length = 2

        def compute(self, today, assets, out, high, low, close):
            out[:] = np.nanmax([(high[-1] - close[0]), (close[0] - low[-1]), (high[-1] - low[-1])], axis=0)
            
    class MACD_Signal_10d(CustomFactor):
        inputs = [USEquityPricing.close]
        window_length = 60

        def compute(self, today, assets, out, close):

            sig_lines = []

            for col in close.T:
                # get signal line only
                try:
                    _, signal_line, _ = talib.MACD(col, fastperiod=12,
                                                   slowperiod=26, signalperiod=10)
                    sig_lines.append(signal_line[-1])
                # if error calculating, return NaN
                except:
                    sig_lines.append(np.nan)
            out[:] = sig_lines
    
    #TRADITIONAL VALUE
    def Dividend_Yield():
        """
        Dividend Yield:
        Dividends per share divided by closing price.
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High Dividend Yield Ratio suggests that an equity
        is attractive to an investor as the dividends
        paid out will be a larger proportion of
        the price they paid for it.
        """
        return vr.dividend_yield.latest
    
    def Price_To_Free_Cashflows():
        """
        Price to Free Cash Flows:
        Closing price divided by free cash flow.
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        Low P/ Free Cash Flows suggests that equity is cheap
        Differs substantially between sectors
        """
        return USEquityPricing.close.latest / \
            vr.fcf_per_share.latest

    def Price_To_Operating_Cashflows():
        """
        Price to Operating Cash Flows:
        Closing price divided by operating cash flow.
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        Low P/ Operating Cash Flows suggests that equity is cheap
        Differs substantially between sectors
        """
        return USEquityPricing.close.latest / \
            vr.cfo_per_share.latest

    def Price_To_Book():
        """
        Price to Book Value:
        Closing price divided by book value.
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        Low P/B Ratio suggests that equity is cheap
        Differs substantially between sectors
        """
        return USEquityPricing.close.latest / \
            vr.book_value_per_share.latest
    
    def EV_To_Cashflows():
        """
        Enterprise Value to Cash Flows:
        Enterprise Value divided by Free Cash Flows.
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        Low EV/FCF suggests that a company has a good amount of
        money relative to its size readily available
        """
        return v.enterprise_value.latest / \
            cfs.free_cash_flow.latest
    
    #Efficiency
    def Capex_To_Sales():
        """
        Capital Expnditure to Sales:
        Capital Expenditure divided by Total Revenue.
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests good efficiency, as expenditure is
        being used to generate greater sales figures
        """
        return (cfs.capital_expenditure.latest * 4.) / \
            (is_.total_revenue.latest * 4.)
    
    def EBIT_To_Assets():
        """
        Earnings Before Interest and Taxes (EBIT) to Total Assets:
        EBIT divided by Total Assets.
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests good efficiency, as earnings are
        being used to generate more assets
        """
        return (is_.ebit.latest * 4.) / \
            bs.total_assets.latest
    
    #Quality
    class Piotroski(CustomFactor):
        inputs = [
            morningstar.operation_ratios.roa,
            morningstar.cash_flow_statement.operating_cash_flow,
            morningstar.cash_flow_statement.cash_flow_from_continuing_operating_activities,

            morningstar.operation_ratios.long_term_debt_equity_ratio,
            morningstar.operation_ratios.current_ratio,
            morningstar.valuation.shares_outstanding,

            morningstar.operation_ratios.gross_margin,
            morningstar.operation_ratios.assets_turnover,
        ]
        window_length = 22

        def compute(self, today, assets, out,
                    roa, cash_flow, cash_flow_from_ops,
                    long_term_debt_ratio, current_ratio, shares_outstanding,
                    gross_margin, assets_turnover):
            profit = (
                (roa[-1] > 0).astype(int) +
                (cash_flow[-1] > 0).astype(int) +
                (roa[-1] > roa[0]).astype(int) +
                (cash_flow_from_ops[-1] > roa[-1]).astype(int)
            )

            leverage = (
                (long_term_debt_ratio[-1] < long_term_debt_ratio[0]).astype(int) +
                (current_ratio[-1] > current_ratio[0]).astype(int) + 
                (shares_outstanding[-1] <= shares_outstanding[0]).astype(int)
            )

            operating = (
                (gross_margin[-1] > gross_margin[0]).astype(int) +
                (assets_turnover[-1] > assets_turnover[0]).astype(int)
            )

            out[:] = profit + leverage + operating


    class ROA(CustomFactor):
        window_length = 1
        inputs = [morningstar.operation_ratios.roa]

        def compute(self, today, assets, out, roa):
            out[:] = (roa[-1] > 0).astype(int)

    class ROAChange(CustomFactor):
        window_length = 22
        inputs = [morningstar.operation_ratios.roa]

        def compute(self, today, assets, out, roa):
            out[:] = (roa[-1] > roa[0]).astype(int)

    class CashFlow(CustomFactor):
        window_length = 1
        inputs = [morningstar.cash_flow_statement.operating_cash_flow]

        def compute(self, today, assets, out, cash_flow):
            out[:] = (cash_flow[-1] > 0).astype(int)

    class CashFlowFromOps(CustomFactor):
        window_length = 1
        inputs = [morningstar.cash_flow_statement.cash_flow_from_continuing_operating_activities, morningstar.operation_ratios.roa]

        def compute(self, today, assets, out, cash_flow_from_ops, roa):
            out[:] = (cash_flow_from_ops[-1] > roa[-1]).astype(int)

    class LongTermDebtRatioChange(CustomFactor):
        window_length = 22
        inputs = [morningstar.operation_ratios.long_term_debt_equity_ratio]

        def compute(self, today, assets, out, long_term_debt_ratio):
            out[:] = (long_term_debt_ratio[-1] < long_term_debt_ratio[0]).astype(int)

    class CurrentDebtRatioChange(CustomFactor):
        window_length = 22
        inputs = [morningstar.operation_ratios.current_ratio]

        def compute(self, today, assets, out, current_ratio):
            out[:] = (current_ratio[-1] > current_ratio[0]).astype(int)

    class SharesOutstandingChange(CustomFactor):
        window_length = 22
        inputs = [morningstar.valuation.shares_outstanding]

        def compute(self, today, assets, out, shares_outstanding):
            out[:] = (shares_outstanding[-1] <= shares_outstanding[0]).astype(int)

    class GrossMarginChange(CustomFactor):
        window_length = 22
        inputs = [morningstar.operation_ratios.gross_margin]

        def compute(self, today, assets, out, gross_margin):
            out[:] = (gross_margin[-1] > gross_margin[0]).astype(int)

    class AssetsTurnoverChange(CustomFactor):
        window_length = 22
        inputs = [morningstar.operation_ratios.assets_turnover]

        def compute(self, today, assets, out, assets_turnover):
            out[:] = (assets_turnover[-1] > assets_turnover[0]).astype(int)
    
    def Asset_Growth_3M():
        """
        3-month Asset Growth:
        Increase in total assets over 3 months
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value represents good financial health as quantity of
        assets is increasing
        """
        return Returns(inputs=[bs.total_assets], window_length=63)
    
    def Asset_To_Equity_Ratio():
        """
        Asset / Equity Ratio
        Total current assets divided by common equity
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests that company has taken on substantial debt
        Vaires substantially with industry
        """
        return bs.total_assets.latest / bs.common_stock_equity.latest
    
    def Dividend_Growth():
        """
        Dividend Growth:
        Growth in dividends observed over a 1-year lookback window
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests that rate at which the quantity of dividends
        paid out is increasing Morningstar built-in fundamental
        better as adjusts inf values
        """
        return morningstar.earnings_ratios.dps_growth.latest
    
    def FCF_EV():
        return factset.Fundamentals.free_cf_fcfe_qf.latest / \
                   factset.Fundamentals.entrpr_val_qf.latest
    
    def DEBT_TOTAL_ASSETS():
        return -factset.Fundamentals.debt.latest / \
                    factset.Fundamentals.assets.latest
    
    class ROIC_GROWTH_2YR(CustomFactor):  
            inputs = [Fundamentals.roic]  
            window_length = 504
            window_safe = True
            def compute(self, today, assets, out, roic):  
                out[:] = (gmean([roic[-1]+1, roic[-252]+1,roic[-504]+1])-1)

    class GM_STABILITY_8YR(CustomFactor):  
            inputs = [Fundamentals.gross_margin]  
            window_length = 9
            window_safe = True
            def compute(self, today, assets, out, gm):  
                out[:] = (gm[-8]) 

    class GM_STABILITY_2YR(CustomFactor):  
            inputs = [Fundamentals.gross_margin]  
            window_length = 504
            window_safe = True
            def compute(self, today, assets, out, gm):  
                out[:] = preprocess(np.std([gm[-1]-gm[-252],gm[-252]-gm[-504]],axis=0)) 
    
    #Profitability
    def Net_Income_Margin():
        """
        Gross Income Margin:
        Gross Profit divided by Net Sales
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests that the company is generating large profits
        Builtin used as cleans inf values
        """
        return or_.net_margin.latest
    
    def Return_On_Total_Equity():
        """
        Return on Total Equity:
        Net income divided by average of total shareholder equity
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests that the company is generating large profits
        Builtin used as cleans inf values
        """
        return or_.roe.latest

    def Return_On_Total_Assets():
        """
        Return on Total Assets:
        Net income divided by average total assets
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests that the company is generating large profits
        Builtin used as cleans inf values
        """
        return or_.roa.latest

    def Return_On_Total_Invest_Capital():
        """
        Return on Total Invest Capital:
        Net income divided by average total invested capital
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests that the company is generating large profits
        Builtin used as cleans inf values
        """
        return or_.roic.latest
    
    #Sentiment
    class MessageSum(CustomFactor):
            inputs = [USEquityPricing.high, USEquityPricing.low, USEquityPricing.close, stocktwits.bull_scored_messages, stocktwits.bear_scored_messages, stocktwits.total_scanned_messages]
            window_length = 21
            window_safe = True
            def compute(self, today, assets, out, high, low, close, bull, bear, total):
                v = np.nansum((high-low)/close, axis=0)
                out[:] = preprocess(v*np.nansum(total*(bear-bull), axis=0))
    
    #Risk
    class Volatility(CustomFactor):

        inputs = [USEquityPricing.close]
        window_length = 252

        def compute(self, today, assets, out, close):  
            close = pd.DataFrame(data=close, columns=assets) 
            # Since we are going to rank largest is best we need to invert the sdev.
            out[:] = 1 / np.log(close).diff().std()

    all_factors = {
        #Momentum
        'Momentum': Momentum,
        'aa_momentum': aa_momentum,
        'ATR': ATR,
        'MINUS_DM': MINUS_DM,
        'PLUS_DM': PLUS_DM,
        'Price Momentum 12M':Price_Momentum_12M,
        #Trend/Direction
        'Trendline': Trendline,
        'LINEARREG_INTERCEPT': LINEARREG_INTERCEPT,
        'TEM': TEM,
        'PriceOscillator': PriceOscillator,
        'MEDPRICE': MEDPRICE,
        'TYPPRICE': TYPPRICE,
        'TRANGE': TRANGE,
        'MACD_Signal_10d': MACD_Signal_10d,
        #TRADITIONAL VALUE
        'Dividend Yield':Dividend_Yield,
        'Price to Free Cashflows': Price_To_Free_Cashflows,
        'Price to Operating Cashflows':Price_To_Operating_Cashflows,
        'Price to Book': Price_To_Book,
        'EV to Cashflows': EV_To_Cashflows,
        #EFFICIENCY
        'Capex to Sales': Capex_To_Sales,
        'EBIT to Assets': EBIT_To_Assets,
        #Quality
        'Piotroski': Piotroski,
        'Asset Growth 3M': Asset_Growth_3M,
        'Asset to Equity Ratio': Asset_To_Equity_Ratio,
        'Dividend Growth': Dividend_Growth,
        'ROIC_GROWTH_2YR': ROIC_GROWTH_2YR,
        'GM_STABILITY_8YR': GM_STABILITY_8YR,
        'GM_STABILITY_2YR': GM_STABILITY_2YR,
        'FCF_EV': FCF_EV,
        'DEBT_TOTAL_ASSETS': DEBT_TOTAL_ASSETS,
        #PROFITABILITY
        'Net Income Margin': Net_Income_Margin,
        'Return on Total Assets': Return_On_Total_Assets,
        'Return on Total Equity': Return_On_Total_Equity,
        'Return on Invest Capital': Return_On_Total_Invest_Capital,
        #Sentiment
        'MessageSum': MessageSum,
        #Risk
        'Volatility': Volatility,
    }     
    
    return all_factors



In [ ]:
def make_factors():
    #Momentum Indicators
    class Momentum(CustomFactor):
        inputs = [USEquityPricing.close]
        window_length = 252

        def compute(self, today, assets, out, close):       
            out[:] = close[-20] / close[0]

    class aa_momentum(CustomFactor):
        """ Alpha Architect - Momentum factor """
        inputs = [USEquityPricing.close,
        Returns(window_length=126)]
        window_length = 252
        def compute(self, today, assets, out, prices, returns):  
                out[:] = ((prices[-21] - prices[-252])/prices[-252] -  
                          (prices[-1] - prices[-21])/prices[-21]) / np.nanstd(returns, axis=0)

    class ATR(CustomFactor):
            """
            Average True Range

            Momentum indicator

            **Default Inputs:** USEquityPricing.high, USEquityPricing.low, USEquityPricing.close

            **Default Window Length:** 15 (14+1)

            https://en.wikipedia.org/wiki/Average_true_range
            """
            inputs=[USEquityPricing.high, USEquityPricing.low, USEquityPricing.close]
            window_length = 21

            def compute(self, today, assets, out, high, low, close):

                tr_frame = trange_helper(high, low, close)
                decay_rate= 2./(len(tr_frame) + 1.)
                weights = np.full(len(tr_frame), decay_rate, float) ** np.arange(len(tr_frame) + 1, 1, -1)
                out[:] = np.average(tr_frame, axis=0, weights=weights)

    class MINUS_DM(CustomFactor):
        """
        Negative directional movement

        Momentum indicator

        **Default Inputs:** USEquityPricing.high, USEquityPricing.low

        **Default Window Length:** 15

        https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/DMI
        """	

        inputs = [USEquityPricing.high, USEquityPricing.low]
        window_length = 22

        def compute(self, today, assets, out, high, low):
                out[:] = np.sum(minus_dm_helper(high, low), axis=0)


    class PLUS_DM(CustomFactor):
        """
        Positive directional movement

        Momentum indicator

        **Default Inputs:** USEquityPricing.high, USEquityPricing.low

        **Default Window Length:** 15

        https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/DMI
        """	
        inputs = [USEquityPricing.high, USEquityPricing.low]
        window_length = 22

        def compute(self, today, assets, out, high, low):
                out[:] = np.sum(plus_dm_helper(high, low), axis=0)

    def Price_Momentum_12M():
        """
        12-Month Price Momentum:
        12-month closing price rate of change.
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests momentum (long term)
        Equivalent to analysis of returns (12-month window)
        """
        return Returns(window_length=252)

    #Trend Indicators
    class Trendline(CustomFactor):
            inputs = [USEquityPricing.close]
            window_length = 252
            window_safe = True

            _x = np.arange(window_length)
            _x_var = np.var(_x)

            def compute(self, today, assets, out, close):

                x_matrix = repeat_last_axis(
                (self.window_length - 1) / 2 - self._x,
                len(assets),
                )

                y_bar = np.nanmean(close, axis=0)
                y_bars = repeat_first_axis(y_bar, self.window_length)
                y_matrix = close - y_bars

                out[:] = preprocess(-np.divide(
                (x_matrix * y_matrix).sum(axis=0) / self._x_var,
                self.window_length
                ))

    class LINEARREG_INTERCEPT(CustomFactor):
        """
        Intercept of Trendline

        **Default Inputs:**  USEquitypricing.close

        **Default Window Length:** 14

        http://www.stat.cmu.edu/~cshalizi/mreg/15/lectures/06/lecture-06.pdf
        """
        inputs=[USEquityPricing.close]
        window_length=21

        # using MLE
        def compute(self, today, assets, out, close):

            # prepare X matrix (x_is - x_bar)
            X = range(self.window_length)
            X_bar = np.nanmean(X)
            X_vector = X - X_bar
            X_matrix = np.tile(X_vector, (len(close.T), 1)).T

            # prepare Y vectors (y_is - y_bar)
            Y_bar = np.nanmean(close, axis=0)
            Y_bars = np.tile(Y_bar, (self.window_length, 1))
            Y_matrix = close - Y_bars

            # prepare variance of X
            X_var = np.nanvar(X)

            # multiply X matrix an Y matrix and sum (dot product)
            # then divide by variance of X
            # this gives the MLE of Beta
            betas = (np.sum((X_matrix * Y_matrix), axis=0) / X_var) / (self.window_length)

            # now use to get to MLE of alpha
            out[:] = Y_bar - (betas * X_bar)

    class MaxGap(CustomFactor): 
        # the biggest absolute overnight gap in the previous 90 sessions
        inputs = [USEquityPricing.close] ; window_length = 90
        window_safe = True
        def compute(self, today, assets, out, close):
            abs_log_rets = np.abs(np.diff(np.log(close),axis=0))
            max_gap = np.max(abs_log_rets, axis=0)
            out[:] = preprocess(max_gap)

    class TEM(CustomFactor):
            """
            TEM = standard deviation of past 6 quarters' reports
            """
            inputs=[factset.Fundamentals.capex_qf_asof_date,
                factset.Fundamentals.capex_qf,
                factset.Fundamentals.assets]
            window_length = 390
            window_safe = True
            def compute(self, today, assets, out, asof_date, capex, total_assets):
                values = capex/total_assets
                values[np.isinf(values)] = np.nan
                out_temp = np.zeros_like(values[-1,:])
                for column_ix in range(asof_date.shape[1]):
                    _, unique_indices = np.unique(asof_date[:, column_ix], return_index=True)
                    quarterly_values = values[unique_indices, column_ix]
                    if len(quarterly_values) < 6:
                        quarterly_values = np.hstack([
                        np.repeat([np.nan], 6 - len(quarterly_values)),
                        quarterly_values,
                        ])

                    out_temp[column_ix] = np.std(quarterly_values[-6:])

                out[:] = preprocess(-out_temp) 

    class PriceOscillator(CustomFactor):
        inputs = (USEquityPricing.close,)
        window_length = 252

        def compute(self, today, assets, out, close):
            four_week_period = close[-20:]
            np.divide(
                np.nanmean(four_week_period, axis=0),
                np.nanmean(close, axis=0),
                out=out,
            )
            out -= 1

    class MeanReversion1M(CustomFactor):
        inputs = (Returns(window_length=21),)
        window_length = 252

        def compute(self, today, assets, out, monthly_rets):
            np.divide(
                monthly_rets[-1] - np.nanmean(monthly_rets, axis=0),
                np.nanstd(monthly_rets, axis=0),
                out=out,
            )

    class ADX(CustomFactor):
        """
        Average Directional Movement Index

        Momentum indicator. Smoothed DX

        **Default Inputs:** USEquityPricing.high, USEquityPricing.low, USEquitypricing.close

        **Default Window Length:** 29

        https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/DMI
        """	    
        inputs = [USEquityPricing.high, USEquityPricing.low, USEquityPricing.close]
        window_length = 29

        def compute(self, today, assets, out, high, low, close):

            # positive directional index
            plus_di = 100 * np.cumsum(plus_dm_helper(high, low) / trange_helper(high, low, close), axis=0)

            # negative directional index
            minus_di = 100 * np.cumsum(minus_dm_helper(high, low) / trange_helper(high, low, close), axis=0)

            # full dx with 15 day burn-in period
            dx_frame = (np.abs(plus_di - minus_di) / (plus_di + minus_di) * 100.)[14:]

            # 14-day EMA
            span = 14.
            decay_rate = 2. / (span + 1.)
            weights = weights_long = np.full(span, decay_rate, float) ** np.arange(span + 1, 1, -1)

            # return EMA
            out[:] = np.average(dx_frame, axis=0, weights=weights)


    class DX(CustomFactor):
        """
        Directional Movement Index

        Momentum indicator

        **Default Inputs:** USEquityPricing.high, USEquityPricing.low, USEquitypricing.close

        **Default Window Length:** 15

        https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/DMI
        """	    
        inputs = [USEquityPricing.high, USEquityPricing.low, USEquityPricing.close]
        window_length = 22

        def compute(self, today, assets, out, high, low, close):

            # positive directional index
            plus_di = 100 * np.sum(plus_dm_helper(high, low) / (trange_helper(high, low, close)), axis=0)

            # negative directional index
            minus_di = 100 * np.sum(minus_dm_helper(high, low) / (trange_helper(high, low, close)), axis=0)

            # DX
            out[:] = np.abs(plus_di - minus_di) / (plus_di + minus_di) * 100.

    class MEDPRICE(CustomFactor):
        """
        Mean of a day's high and low prices

        **Default Inputs:**  USEquityPricing.high, USEquityPricing.low

        **Default Window Length:** 1

        http://www.fmlabs.com/reference/default.htm?url=MedianPrices.htm
        """	    
        inputs = [USEquityPricing.high, USEquityPricing.low]
        window_length = 1

        def compute(self, today, assets, out, high, low):
            out[:] = (high + low) / 2.

    class TYPPRICE(CustomFactor):
        """
        Typical Price 

        **Default Inputs:**  USEquityPricing.high, USEquityPricing.low, USEquityPricing.close

        **Default Window Length:** 1

        https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/typical-price
        """    
        inputs = [USEquityPricing.high, USEquityPricing.low, USEquityPricing.close]
        window_length = 1

        def compute(self, today, assets, out, high, low, close):
            out[:] = (high + low + close) / 3.


    class TRANGE(CustomFactor):
        """
        True Range 

        **Default Inputs:**  USEquityPricing.high, USEquityPricing.low, USEquityPricing.close

        **Default Window Length:** 2

        https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/atr
        """    
        inputs = [USEquityPricing.high, USEquityPricing.low, USEquityPricing.close]
        window_length = 2

        def compute(self, today, assets, out, high, low, close):
            out[:] = np.nanmax([(high[-1] - close[0]), (close[0] - low[-1]), (high[-1] - low[-1])], axis=0)

    class MACD_Signal_10d(CustomFactor):
        inputs = [USEquityPricing.close]
        window_length = 60

        def compute(self, today, assets, out, close):

            sig_lines = []

            for col in close.T:
                # get signal line only
                try:
                    _, signal_line, _ = talib.MACD(col, fastperiod=12,
                                                   slowperiod=26, signalperiod=10)
                    sig_lines.append(signal_line[-1])
                # if error calculating, return NaN
                except:
                    sig_lines.append(np.nan)
            out[:] = sig_lines

    #TRADITIONAL VALUE
    def Dividend_Yield():
        """
        Dividend Yield:
        Dividends per share divided by closing price.
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High Dividend Yield Ratio suggests that an equity
        is attractive to an investor as the dividends
        paid out will be a larger proportion of
        the price they paid for it.
        """
        return vr.dividend_yield.latest

    def Price_To_Free_Cashflows():
        """
        Price to Free Cash Flows:
        Closing price divided by free cash flow.
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        Low P/ Free Cash Flows suggests that equity is cheap
        Differs substantially between sectors
        """
        return USEquityPricing.close.latest / \
            vr.fcf_per_share.latest

    def Price_To_Operating_Cashflows():
        """
        Price to Operating Cash Flows:
        Closing price divided by operating cash flow.
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        Low P/ Operating Cash Flows suggests that equity is cheap
        Differs substantially between sectors
        """
        return USEquityPricing.close.latest / \
            vr.cfo_per_share.latest

    def Price_To_Book():
        """
        Price to Book Value:
        Closing price divided by book value.
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        Low P/B Ratio suggests that equity is cheap
        Differs substantially between sectors
        """
        return USEquityPricing.close.latest / \
            vr.book_value_per_share.latest

    def EV_To_Cashflows():
        """
        Enterprise Value to Cash Flows:
        Enterprise Value divided by Free Cash Flows.
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        Low EV/FCF suggests that a company has a good amount of
        money relative to its size readily available
        """
        return v.enterprise_value.latest / \
            cfs.free_cash_flow.latest

    #Efficiency
    def Capex_To_Sales():
        """
        Capital Expnditure to Sales:
        Capital Expenditure divided by Total Revenue.
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests good efficiency, as expenditure is
        being used to generate greater sales figures
        """
        return (cfs.capital_expenditure.latest * 4.) / \
            (is_.total_revenue.latest * 4.)

    def EBIT_To_Assets():
        """
        Earnings Before Interest and Taxes (EBIT) to Total Assets:
        EBIT divided by Total Assets.
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests good efficiency, as earnings are
        being used to generate more assets
        """
        return (is_.ebit.latest * 4.) / \
            bs.total_assets.latest

    #Quality
    class Piotroski(CustomFactor):
        inputs = [
            morningstar.operation_ratios.roa,
            morningstar.cash_flow_statement.operating_cash_flow,
            morningstar.cash_flow_statement.cash_flow_from_continuing_operating_activities,

            morningstar.operation_ratios.long_term_debt_equity_ratio,
            morningstar.operation_ratios.current_ratio,
            morningstar.valuation.shares_outstanding,

            morningstar.operation_ratios.gross_margin,
            morningstar.operation_ratios.assets_turnover,
        ]
        window_length = 22

        def compute(self, today, assets, out,
                    roa, cash_flow, cash_flow_from_ops,
                    long_term_debt_ratio, current_ratio, shares_outstanding,
                    gross_margin, assets_turnover):
            profit = (
                (roa[-1] > 0).astype(int) +
                (cash_flow[-1] > 0).astype(int) +
                (roa[-1] > roa[0]).astype(int) +
                (cash_flow_from_ops[-1] > roa[-1]).astype(int)
            )

            leverage = (
                (long_term_debt_ratio[-1] < long_term_debt_ratio[0]).astype(int) +
                (current_ratio[-1] > current_ratio[0]).astype(int) + 
                (shares_outstanding[-1] <= shares_outstanding[0]).astype(int)
            )

            operating = (
                (gross_margin[-1] > gross_margin[0]).astype(int) +
                (assets_turnover[-1] > assets_turnover[0]).astype(int)
            )

            out[:] = profit + leverage + operating


    class ROA(CustomFactor):
        window_length = 1
        inputs = [morningstar.operation_ratios.roa]

        def compute(self, today, assets, out, roa):
            out[:] = (roa[-1] > 0).astype(int)

    class ROAChange(CustomFactor):
        window_length = 22
        inputs = [morningstar.operation_ratios.roa]

        def compute(self, today, assets, out, roa):
            out[:] = (roa[-1] > roa[0]).astype(int)

    class CashFlow(CustomFactor):
        window_length = 1
        inputs = [morningstar.cash_flow_statement.operating_cash_flow]

        def compute(self, today, assets, out, cash_flow):
            out[:] = (cash_flow[-1] > 0).astype(int)

    class CashFlowFromOps(CustomFactor):
        window_length = 1
        inputs = [morningstar.cash_flow_statement.cash_flow_from_continuing_operating_activities, morningstar.operation_ratios.roa]

        def compute(self, today, assets, out, cash_flow_from_ops, roa):
            out[:] = (cash_flow_from_ops[-1] > roa[-1]).astype(int)

    class LongTermDebtRatioChange(CustomFactor):
        window_length = 22
        inputs = [morningstar.operation_ratios.long_term_debt_equity_ratio]

        def compute(self, today, assets, out, long_term_debt_ratio):
            out[:] = (long_term_debt_ratio[-1] < long_term_debt_ratio[0]).astype(int)

    class CurrentDebtRatioChange(CustomFactor):
        window_length = 22
        inputs = [morningstar.operation_ratios.current_ratio]

        def compute(self, today, assets, out, current_ratio):
            out[:] = (current_ratio[-1] > current_ratio[0]).astype(int)

    class SharesOutstandingChange(CustomFactor):
        window_length = 22
        inputs = [morningstar.valuation.shares_outstanding]

        def compute(self, today, assets, out, shares_outstanding):
            out[:] = (shares_outstanding[-1] <= shares_outstanding[0]).astype(int)

    class GrossMarginChange(CustomFactor):
        window_length = 22
        inputs = [morningstar.operation_ratios.gross_margin]

        def compute(self, today, assets, out, gross_margin):
            out[:] = (gross_margin[-1] > gross_margin[0]).astype(int)

    class AssetsTurnoverChange(CustomFactor):
        window_length = 22
        inputs = [morningstar.operation_ratios.assets_turnover]

        def compute(self, today, assets, out, assets_turnover):
            out[:] = (assets_turnover[-1] > assets_turnover[0]).astype(int)

    def Asset_Growth_3M():
        """
        3-month Asset Growth:
        Increase in total assets over 3 months
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value represents good financial health as quantity of
        assets is increasing
        """
        return Returns(inputs=[bs.total_assets], window_length=63)

    def Asset_To_Equity_Ratio():
        """
        Asset / Equity Ratio
        Total current assets divided by common equity
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests that company has taken on substantial debt
        Vaires substantially with industry
        """
        return bs.total_assets.latest / bs.common_stock_equity.latest

    def Debt_To_Asset_Ratio():
        """
        Debt / Asset Ratio:
        Total Debts divided by Total Assets
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests that company has taken on substantial debt
        Low value suggests good financial health as assets greater than debt
        Long Term Debt
        """
        return bs.total_debt.latest / bs.total_assets.latest

    def Dividend_Growth():
        """
        Dividend Growth:
        Growth in dividends observed over a 1-year lookback window
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests that rate at which the quantity of dividends
        paid out is increasing Morningstar built-in fundamental
        better as adjusts inf values
        """
        return morningstar.earnings_ratios.dps_growth.latest

    def FCF_EV():
        return factset.Fundamentals.free_cf_fcfe_qf.latest / \
                   factset.Fundamentals.entrpr_val_qf.latest

    def DEBT_TOTAL_ASSETS():
        return -factset.Fundamentals.debt.latest / \
                    factset.Fundamentals.assets.latest

    class ROIC_GROWTH_2YR(CustomFactor):  
            inputs = [Fundamentals.roic]  
            window_length = 504
            window_safe = True
            def compute(self, today, assets, out, roic):  
                out[:] = (gmean([roic[-1]+1, roic[-252]+1,roic[-504]+1])-1)

    class GM_STABILITY_8YR(CustomFactor):  
            inputs = [Fundamentals.gross_margin]  
            window_length = 9
            window_safe = True
            def compute(self, today, assets, out, gm):  
                out[:] = (gm[-8]) 

    class GM_STABILITY_2YR(CustomFactor):  
            inputs = [Fundamentals.gross_margin]  
            window_length = 504
            window_safe = True
            def compute(self, today, assets, out, gm):  
                out[:] = preprocess(np.std([gm[-1]-gm[-252],gm[-252]-gm[-504]],axis=0)) 

    #Profitability
    def Net_Income_Margin():
        """
        Gross Income Margin:
        Gross Profit divided by Net Sales
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests that the company is generating large profits
        Builtin used as cleans inf values
        """
        return or_.net_margin.latest

    def Return_On_Total_Equity():
        """
        Return on Total Equity:
        Net income divided by average of total shareholder equity
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests that the company is generating large profits
        Builtin used as cleans inf values
        """
        return or_.roe.latest

    def Return_On_Total_Assets():
        """
        Return on Total Assets:
        Net income divided by average total assets
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests that the company is generating large profits
        Builtin used as cleans inf values
        """
        return or_.roa.latest

    def Return_On_Total_Invest_Capital():
        """
        Return on Total Invest Capital:
        Net income divided by average total invested capital
        https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf # NOQA
        Notes:
        High value suggests that the company is generating large profits
        Builtin used as cleans inf values
        """
        return or_.roic.latest

    #Sentiment
    class MessageSum(CustomFactor):
            inputs = [USEquityPricing.high, USEquityPricing.low, USEquityPricing.close, stocktwits.bull_scored_messages, stocktwits.bear_scored_messages, stocktwits.total_scanned_messages]
            window_length = 21
            window_safe = True
            def compute(self, today, assets, out, high, low, close, bull, bear, total):
                v = np.nansum((high-low)/close, axis=0)
                out[:] = preprocess(v*np.nansum(total*(bear-bull), axis=0))

    #Risk
    class Volatility(CustomFactor):

        inputs = [USEquityPricing.close]
        window_length = 252

        def compute(self, today, assets, out, close):  
            close = pd.DataFrame(data=close, columns=assets) 
            # Since we are going to rank largest is best we need to invert the sdev.
            out[:] = 1 / np.log(close).diff().std()

    class Volatility3M(CustomFactor):
        inputs = [Returns(window_length=2)]
        window_length = 63

        def compute(self, today, assets, out, rets):
            np.nanstd(rets, axis=0, out=out) 

    all_factors = {
        #Momentum
        'Momentum': Momentum,
        'aa_momentum': aa_momentum,
        'ATR': ATR,
        'MINUS_DM': MINUS_DM,
        'PLUS_DM': PLUS_DM,
        'Price Momentum 12M':Price_Momentum_12M,
        #Trend/Direction
        'Trendline': Trendline,
        'LINEARREG_INTERCEPT': LINEARREG_INTERCEPT,
        'MaxGap': MaxGap,
        'TEM': TEM,
        'PriceOscillator': PriceOscillator,
        'MeanReversion1M': MeanReversion1M,
        'ADX': ADX,
        'DX': DX,
        'MEDPRICE': MEDPRICE,
        'TYPPRICE': TYPPRICE,
        'TRANGE': TRANGE,
        'MACD_Signal_10d': MACD_Signal_10d,
        #TRADITIONAL VALUE
        'Dividend Yield':Dividend_Yield,
        'Price to Free Cashflows': Price_To_Free_Cashflows,
        'Price to Operating Cashflows':Price_To_Operating_Cashflows,
        'Price to Book': Price_To_Book,
        'EV to Cashflows': EV_To_Cashflows,
        #EFFICIENCY
        'Capex to Sales': Capex_To_Sales,
        'EBIT to Assets': EBIT_To_Assets,
        #Quality
        'Piotroski': Piotroski,
        'Asset Growth 3M': Asset_Growth_3M,
        'Asset to Equity Ratio': Asset_To_Equity_Ratio,
        'Debt to Asset Ratio': Debt_To_Asset_Ratio,
        'Dividend Growth': Dividend_Growth,
        'ROIC_GROWTH_2YR': ROIC_GROWTH_2YR,
        'GM_STABILITY_8YR': GM_STABILITY_8YR,
        'GM_STABILITY_2YR': GM_STABILITY_2YR,
        'FCF_EV': FCF_EV,
        'DEBT_TOTAL_ASSETS': DEBT_TOTAL_ASSETS,
        #PROFITABILITY
        'Net Income Margin': Net_Income_Margin,
        'Return on Total Assets': Return_On_Total_Assets,
        'Return on Total Equity': Return_On_Total_Equity,
        'Return on Invest Capital': Return_On_Total_Invest_Capital,
        #Sentiment
        'MessageSum': MessageSum,
        #Risk
        'Volatility': Volatility,
        'Volatility3M': Volatility3M,
    }        

    return all_factors



## Define universe and select factors to use

In [ ]:
universe = QTradableStocksUS()

factors = make_factors()

## Define and build the pipeline
Next we have to build the pipeline. In addition to the factors defined above, we need the forward returns we want to predict. In this Notebook we will predict 5-day returns and train our model on daily data. You can also subsample the data to e.g. weekly to not have overlapping return periods but we omit this here.

In [ ]:
n_fwd_days = 21 # number of days to compute returns over

In [ ]:
def make_history_pipeline(factors, universe, n_fwd_days=21):
    # Call .rank() on all factors and mask out the universe
    factor_zscored = {name: f().zscore(mask=universe) for name, f in factors.iteritems()}
    # Get cumulative returns over last n_fwd_days days. We will later shift these.
    factor_zscored['Returns'] = Returns(inputs=[USEquityPricing.open],
                                      mask=universe, window_length=n_fwd_days)
    
    pipe = Pipeline(screen=universe, columns=factor_zscored)
    
    return pipe

In [ ]:
history_pipe = make_history_pipeline(factors, universe, n_fwd_days=n_fwd_days)

## Run the pipeline

In [ ]:
start = pd.Timestamp("2015-01-01")
end = pd.Timestamp("2016-01-01")
results = run_pipeline(history_pipe, start_date=start, end_date=end)
results.index.names = ['date', 'security']
results.head()

Next, we are doing some additional transformations to our data:

1. Shift factor ranks to align with future returns `n_fwd_days` days in the future.
2. Find the top and bottom 30 percentile stocks by their returns. Essentially, we only care about relative movement of stocks. If we later short stocks that go down and long stocks going up relative to each other, it doesn't matter if e.g. all stocks are going down in absolute terms. Moverover, we are ignoring stocks that did not move that much (i.e. 30th to 70th percentile) to only train the classifier on those that provided strong signal. 
3. We also binarize the returns by their percentile to turn our ML problem into a classification one.

`shift_mask_data()` is a utility function that does all of these.

In [ ]:
def shift_mask_data(X, Y, upper_percentile=70, lower_percentile=30, n_fwd_days=1):
    # Shift X to match factors at t to returns at t+n_fwd_days (we want to predict future returns after all)
    shifted_X = np.roll(X, n_fwd_days+1, axis=0)
    
    # Slice off rolled elements
    X = shifted_X[n_fwd_days+1:]
    Y = Y[n_fwd_days+1:]
    
    n_time, n_stocks, n_factors = X.shape
    
    # Look for biggest up and down movers
    upper = np.nanpercentile(Y, upper_percentile, axis=1)[:, np.newaxis]
    lower = np.nanpercentile(Y, lower_percentile, axis=1)[:, np.newaxis]
  
    upper_mask = (Y >= upper)
    lower_mask = (Y <= lower)
    
    mask = upper_mask | lower_mask # This also drops nans
    mask = mask.flatten()
    
    # Only try to predict whether a stock moved up/down relative to other stocks
    Y_binary = np.zeros(n_time * n_stocks)
    Y_binary[upper_mask.flatten()] = 1
    Y_binary[lower_mask.flatten()] = -1
    
    # Flatten X
    X = X.reshape((n_time * n_stocks, n_factors))

    # Drop stocks that did not move much (i.e. are in the 30th to 70th percentile)
    X = X[mask]
    Y_binary = Y_binary[mask]
    
    return X, Y_binary

In [ ]:
# Massage data to be in the form expected by shift_mask_data()
results_wo_returns = results.copy()
returns = results_wo_returns.pop('Returns')
Y = returns.unstack().values
X = results_wo_returns.to_panel() 
X = X.swapaxes(2, 0).swapaxes(0, 1).values # (factors, time, stocks) -> (time, stocks, factors)

In [ ]:
results_wo_returns.index = results_wo_returns.index.set_levels(results_wo_returns.index.get_level_values(1).map(lambda x: x.symbol), 1, )

In [ ]:
results_wo_returns.index = results_wo_returns.index.set_levels(results_wo_returns.index.get_level_values(0).map(lambda x: x.date), 0, )

In [ ]:
results_wo_returns.sample(10).sort()

In [ ]:
tmp = (returns > 0.).to_frame()
tmp.index = tmp.index.set_levels(tmp.index.get_level_values(1).map(lambda x: x.symbol), 1)
tmp.columns = ['21-day forward returns > 0']
tmp.sample(10).sort()

# Machine Learning

In [ ]:
# Train-test split
train_size_perc = 0.8
n_time, n_stocks, n_factors = X.shape
train_size = np.int16(np.round(train_size_perc * n_time))
X_train, Y_train = X[:train_size, ...], Y[:train_size]
X_test, Y_test = X[(train_size+n_fwd_days):, ...], Y[(train_size+n_fwd_days):]

In [ ]:
X_train_shift, Y_train_shift = shift_mask_data(X_train, Y_train, n_fwd_days=n_fwd_days)
X_test_shift, Y_test_shift = shift_mask_data(X_test, Y_test, n_fwd_days=n_fwd_days, 
                                             lower_percentile=50, 
                                             upper_percentile=50)

In [ ]:
X_train_shift.shape, X_test_shift.shape

## Training our ML pipeline
Before training our classifier, several preprocessing steps are advisable. The first one imputes nan values with the factor mean to get clean training data, the second scales our factor ranks to be between [0, 1).

For training we are using the [AdaBoost classifier](https://en.wikipedia.org/wiki/AdaBoost) which automatically determines the most relevant features (factors) and tries to find a non-linear combination of features to maximize predictiveness while still being robust. In essence, AdaBoost trains an ensemble of weak classifiers (decision trees in this case) sequentially. Each subsequent weak classifier takes into account the samples (or data points) already classified by the previous weak classifiers. It then focuses on the samples misclassified by the previous weak classifiers and tries to get those correctly. With each new weak classifier you get more fine-grained in your decision function and correctly classify some previously misclassified samples. For prediction, you simply average the answer of all weak classifiers to get a single strong classifier.

Of course, this is just an example and you can let your creativity and skill roam freely.

In [ ]:
# Train classifier
imputer = preprocessing.Imputer()
scaler = preprocessing.MinMaxScaler()
clf = ensemble.AdaBoostClassifier(n_estimators=150) # n_estimators controls how many weak classifiers are fi

X_train_trans = imputer.fit_transform(X_train_shift)
X_train_trans = scaler.fit_transform(X_train_trans)
clf.fit(X_train_trans, Y_train_shift)


In [ ]:
Y_pred = clf.predict(X_train_trans)
print('Accuracy on train set = {:.2f}%'.format(metrics.accuracy_score(Y_train_shift, Y_pred) * 100))

## Evaluating our ML classifier
To evaluate our ML classifer on the test set, we have to transform our test data in the same way as our traning data. Note that we are only calling the `.transform()` method here which does not use any information from the test set.

In [ ]:
# Transform test data
X_test_trans = imputer.transform(X_test_shift)
X_test_trans = scaler.transform(X_test_trans)

In [ ]:
# Predict!
Y_pred = clf.predict(X_test_trans)

Y_pred_prob = clf.predict_proba(X_test_trans)

In [ ]:
print 'Predictions:', Y_pred
print 'Probabilities of class == 1:', Y_pred_prob[:, 1] * 100

In [ ]:
print('Accuracy on test set = {:.2f}%'.format(metrics.accuracy_score(Y_test_shift, Y_pred) * 100))
print('Log-loss = {:.5f}'.format(metrics.log_loss(Y_test_shift, Y_pred_prob)))

In [ ]:
print results_wo_returns.columns

In [ ]:
feature_importances = pd.Series(clf.feature_importances_)
feature_importances.sort(ascending=False)
ax = feature_importances.plot(kind='bar')
ax.set(ylabel='Importance (Gini Coefficient)', title='Feature importances');

# PART II

## Define the Machine Learning Pipeline Factor
Where before we called `run_pipeline()` to get our data and then train a ML model on the `DataFrame` we received, we now move this step into a Pipeline Factor as well. The steps taken are the same, first we make the factor values match up with the future returns, we binarize the returns, and then impute and scale. Note that we retrain the model only weekly but certainly that's a choice you have to make.

The same factor then also does prediction. For that we call the `.predict_proba()` method which returns a probability of the stock going up. Ultimately, this output will feed into the next stage: portfolio construction. 

In [ ]:
def shift_mask_data(X, Y, upper_percentile=70, lower_percentile=30, n_fwd_days=1):
    # Shift X to match factors at t to returns at t+n_fwd_days (we want to predict future returns after all)
    shifted_X = np.roll(X, n_fwd_days+1, axis=0)
    
    # Slice off rolled elements
    X = shifted_X[n_fwd_days+1:]
    Y = Y[n_fwd_days+1:]
    
    n_time, n_stocks, n_factors = X.shape
    
    # Look for biggest up and down movers
    upper = np.nanpercentile(Y, upper_percentile, axis=1)[:, np.newaxis]
    lower = np.nanpercentile(Y, lower_percentile, axis=1)[:, np.newaxis]
  
    upper_mask = (Y >= upper)
    lower_mask = (Y <= lower)
    
    mask = upper_mask | lower_mask # This also drops nans
    mask = mask.flatten()
    
    # Only try to predict whether a stock moved up/down relative to other stocks
    Y_binary = np.zeros(n_time * n_stocks)
    Y_binary[upper_mask.flatten()] = 1
    Y_binary[lower_mask.flatten()] = -1
    
    # Flatten X
    X = X.reshape((n_time * n_stocks, n_factors))

    # Drop stocks that did not move much (i.e. are in the 30th to 70th percentile)
    X = X[mask]
    Y_binary = Y_binary[mask]
    
    return X, Y_binary

def get_last_values(input_data):
    last_values = []
    for dataset in input_data:
        last_values.append(dataset[-1])
    return np.vstack(last_values).T

In [ ]:
class ML(CustomFactor):
    init = False

    def compute(self, today, assets, out, returns, *inputs):
        # inputs is a list of factors, for example, assume we have 2 alpha signals, 3 stocks,
        # and a lookback of 2 days. Each element in the inputs list will be data of
        # one signal, so len(inputs) == 2. Then each element will contain a 2-D array
        # of shape [time x stocks]. For example:
        # inputs[0]:
        # [[1, 3, 2], # factor 1 rankings of day t-1 for 3 stocks  
        #  [3, 2, 1]] # factor 1 rankings of day t for 3 stocks
        # inputs[1]:
        # [[2, 3, 1], # factor 2 rankings of day t-1 for 3 stocks
        #  [1, 2, 3]] # factor 2 rankings of day t for 3 stocks
        
        if (not self.init) or (today.weekday() == 0): # Monday
            # Instantiate sklearn objects
            self.imputer = preprocessing.Imputer()
            self.scaler = preprocessing.MinMaxScaler()
            self.clf = ensemble.AdaBoostClassifier(n_estimators=100)
            
            # Stack factor rankings
            X = np.dstack(inputs) # (time, stocks, factors)
            Y = returns # (time, stocks)
        
            # Shift data to match with future returns and binarize 
            # returns based on their 
            X, Y = shift_mask_data(X, Y, n_fwd_days=n_fwd_days)
            
            X = self.imputer.fit_transform(X)            
            X = self.scaler.fit_transform(X)
            
            # Fit the classifier
            self.clf.fit(X, Y)
            
            self.init = True

        # Predict
        # Get most recent factor values (inputs always has the full history)
        last_factor_values = get_last_values(inputs)
        last_factor_values = self.imputer.transform(last_factor_values)
        last_factor_values = self.scaler.transform(last_factor_values)

        # Predict the probability for each stock going up 
        # (column 2 of the output of .predict_proba()) and
        # return it via assignment to out.
        out[:] = self.clf.predict_proba(last_factor_values)[:, 1]

In [ ]:
from collections import OrderedDict

def make_ml_pipeline(factors, universe, window_length=63, n_fwd_days=21):
    factors_pipe = OrderedDict()
    # Create returns over last n days.
    factors_pipe['Returns'] = Returns(inputs=[USEquityPricing.open],
                                      mask=universe, window_length=n_fwd_days)
    # Instantiate ranked factors
    for name, f in factors.iteritems():
        factors_pipe[name] = f().rank(mask=universe)
        
    # Create our ML pipeline factor. The window_length will control how much
    # lookback the passed in data will have.
    factors_pipe['ML'] = ML(inputs=factors_pipe.values(), 
                            window_length=window_length + 1, 
                            mask=universe)
    
    pipe = Pipeline(screen=universe, columns=factors_pipe)
    
    return pipe

In [ ]:
ml_pipe = make_ml_pipeline(factors, universe)

In [ ]:
start = pd.Timestamp("2015-01-01") # Can't choose a much longer time-period or we run out of RAM
end = pd.Timestamp("2016-03-01")

results = run_pipeline(ml_pipe, start_date=start, end_date=end)

What happened here?

Our complete pipeline was run every day with the current data (look-ahead bias free). Every Monday we preprocessed the data and trained a classifier. We then used that classifier for predictions on subsequent days.

The 'ML' column will contain for each day, the predicted probabilities of each stock to go up, relative to the other ones in the universe. From here we could create a portfolio inside of an algorithm and trade into it. We will show this step in a future post. Lets examine what the output looks like:

In [ ]:
results['ML'].head()

In [ ]:
results['ML'].tail()

## Analyzing our mega-alpha with pipeline
It's important to realize that our predictions are just another alpha signal (one that is hopefully more predictive than each individual signal in isolation). Because of that, we can analyze just like any other alpha signal with [AlphaLens](https://www.quantopian.com/posts/alphalens-a-new-tool-for-analyzing-alpha-factors).

First, we need to get the pricing data.

In [ ]:
assets = results.index.levels[1]
pricing = get_pricing(assets, start, end + pd.Timedelta(days=252), fields="open_price")

In [ ]:
import alphalens
from alphalens.utils import get_clean_factor_and_forward_returns
from alphalens.tears import create_full_tear_sheet

factor_data = alphalens.utils.get_clean_factor_and_forward_returns(results['ML'], pricing)
alphalens.tears.create_full_tear_sheet(factor_data)